In [1]:
import numpy as np
import tensorflow as tf

# 1 data

In [2]:
mnist = tf.keras.datasets.mnist

In [21]:
(x_train_src,y_train_src),(x_test_src,y_test_src) = mnist.load_data('./mnist.npz')

In [22]:
x_train_src.shape

(60000, 28, 28)

## 2 feature

In [226]:
x_train = x_train_src/255.
x_test = x_test_src/255.

In [227]:
num_class = len(set(y_train_src))
print(num_class)
print(set(y_train_src))

10
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [228]:
def ont_hot(y_train_src,num_class):
    '''
    not in-line process
    '''
    y_train = list()
    y_train_copy = y_train_src.copy()
    for i in range(y_train_copy.shape[0]):
        index = y_train_copy[i]
        row = np.zeros(num_class)
        row[index] = 1
        y_train.append(row)
    y_train = np.array(y_train)
    assert(y_train.shape[0] == y_train_src.shape[0])
    return y_train

In [229]:
y_train = ont_hot(y_train_src,num_class)
y_test = ont_hot(y_test_src,num_class)

In [230]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(60000, 28, 28) (60000, 10)
(10000, 28, 28) (10000, 10)


# 3 model

In [231]:
(m,n) = x_train.shape[1:];
print(m,n)

28 28


In [232]:
x = tf.placeholder(dtype=tf.float32,shape=[None,m,n])
y = tf.placeholder(dtype=tf.int16,shape=[None,num_class])

In [233]:
#x_flag = tf.reshape?

In [234]:
x_flat = tf.reshape(x,shape=[-1,m*n],name='flat')

In [235]:
def full_connect(input_x,n_hidden,name,activation = None):
    '''
    name_score and name for get_variable
    '''
    with tf.name_scope(name):
        i_dim = int(input_x.shape[1])
        stddev = 2./np.sqrt(i_dim)
        w = tf.Variable(tf.truncated_normal(shape=[i_dim,n_hidden],stddev=stddev),name='weight')
        b = tf.Variable(tf.zeros([n_hidden]),dtype=tf.float32)
        output = tf.matmul(input_x,w) + b
        if activation == 'relu':
            activ = tf.nn.relu(output)
        elif activation == 'sigmoid':
            activ = tf.nn.sigmoid(output)
        elif activation == 'tanh':
            activ = tf.nn.tanh(output)
        else  :
            activ = output
        return activ

In [236]:
hidden1 = 100
hidden2 = 200
output = 10

In [237]:
h_1 = full_connect(x_flat,hidden1,'h1',activation='relu')## 换成sigmoid效果很差很多
h_2 = full_connect(h_1,hidden2,'h2',activation='relu')
output = full_connect(h_2,output,'output')

# 4 loss

In [238]:
#tf.nn.softmax_cross_entropy_with_logits?
#label可以是softmax分布

In [239]:
#tf.nn.sparse_softmax_cross_entropy_with_logits?

In [240]:
all_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=output,name='all_loss')

In [241]:
all_loss.shape

TensorShape([Dimension(None)])

In [242]:
loss = tf.reduce_mean(all_loss,name='loss')

In [243]:
loss.shape

TensorShape([])

# 5 optimize

In [244]:
optimize = tf.train.GradientDescentOptimizer(learning_rate=1e-2)
train_op = optimize.minimize(loss)

In [245]:
#correct = tf.nn.top_k?

In [246]:
acc = tf.reduce_mean(tf.cast(tf.equal(tf.nn.top_k(output,name='k1')[1], tf.nn.top_k(y,name='k2')[1]),float),name = 'acc')

# 6 init

In [247]:
init_op = tf.global_variables_initializer()

# 7 train

In [248]:
saver = tf.train.Saver()## for save and restore

In [249]:
#saver.save?

In [250]:
def fetch_batch(n_epochs,batch_index,batch_size,x_train,y_train):
    beg = (batch_index -1)*batch_size
    end = batch_index *batch_size
    return x_train[beg:end],y_train[beg:end]

In [251]:
num_epochs = 6
batch_size = 128
num_batch = int(x_train.shape[0]/batch_size) + 1

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(num_epochs):
        for j in range(1,num_batch):
            sample_x,sample_y = fetch_batch(i,j,batch_size,x_train,y_train)
            if j% 100 == 0:
                acc_train,_ ,loss_val,= sess.run([acc,train_op,loss],feed_dict={x:sample_x,y:sample_y})
                acc_test,= sess.run([acc],feed_dict={x:x_test,y:y_test})
                print('%d\t%d\t%f\t%f\t%f'%(i,j,loss_val,acc_train,acc_test))
            else:
                sess.run([train_op],feed_dict={x:sample_x,y:sample_y})
                
    acc_train,loss_val = sess.run([acc,loss],feed_dict={x:x_train,y:y_train})
    acc_test, = sess.run([acc],feed_dict={x:x_test,y:y_test})
    
    print('%f\t%f\t%f'%(loss_val,acc_train,acc_test))
    saver.save(sess,'./model/mlp',global_step=i*j)

0	100	1.239328	0.664062	0.709800
0	200	0.664090	0.828125	0.812900
0	300	0.681339	0.789062	0.849900
0	400	0.518813	0.843750	0.867100
1	100	0.519469	0.835938	0.883000
1	200	0.366485	0.898438	0.890200
1	300	0.434423	0.828125	0.895500
1	400	0.361221	0.890625	0.900300
2	100	0.420835	0.867188	0.904000
2	200	0.318010	0.890625	0.908500
2	300	0.354090	0.882812	0.912200
2	400	0.298957	0.890625	0.913700
3	100	0.376177	0.890625	0.915600
3	200	0.296010	0.906250	0.916400
3	300	0.313614	0.898438	0.920000
3	400	0.259571	0.898438	0.921100
4	100	0.349147	0.906250	0.923000


In [ ]:
#saver.restore?

# 8 eval test

In [ ]:
with tf.Session() as sess:
    saver.restore(sess,'./model/mlp-2340')
    accuary,loss_val,= sess.run([acc,loss],feed_dict={x:x_test,y:y_test})
    print('%f\t%f'%(loss_val,accuary))